In [1]:
import os
os.chdir('..')

%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from bs4 import BeautifulSoup

In [121]:
with open('data/processed/port_wsc.html', 'r') as f:
    soup = BeautifulSoup(f, 'html5lib')

rows = []
for item in soup.find('ol').find_all('li', recursive=False):
    content = [it.strip().replace('  ', ' ') for it in item.text.split('\n') if it.strip() != '' ]
    schema = (content[-6] + ' ' + content[-5])
    snippet = content[-4].replace('Trecho: ', '')
    if schema[-1] == '.' and schema[-1] != snippet[-1]:
        snippet += schema[-1]
    pronoun = item.find('b').text.strip()
    correct_answer = content[-1].replace('Resposta Correta: ', '').strip()[0]
    substitution_a = content[-3]
    substitution_b = content[-2]
    row = [schema, snippet, pronoun, correct_answer, substitution_a, substitution_b]
    rows.append(row)

df=pd.DataFrame(rows, columns=['schema', 'snippet', 'pronoun', 'correct_answer', 'substitution_a', 'substitution_b'])

In [81]:
row = df.iloc[0]
len_before_pronoun = len(row.schema.replace(row.snippet, ''))
len_after_pronoun = len(row.snippet.replace(row.pronoun, ''))
snippet_a = row.schema[:len_before_pronoun-1] + row.substitution_a.lower() + row.schema[-len_after_pronoun-1:]
snippet_b = row.schema[:len_before_pronoun-1] + row.substitution_b.lower() + row.schema[-len_after_pronoun-1:]

print(snippet_a)
snippet_b

Os vereadores recusaram a autorização aos manifestantes porque os vereadores temiam a violência.


'Os vereadores recusaram a autorização aos manifestantes porque os manifestantes temiam a violência.'

In [132]:
def generate_full_sentences(row):
    len_before_pronoun = len(row.schema.replace(row.snippet, ''))
    len_after_pronoun = len(row.snippet.replace(row.pronoun, ''))
    # TODO names shouldn't be lowercased
    snippet_a = row.schema[:len_before_pronoun] + row.substitution_a.lower() + row.schema[-len_after_pronoun:]
    snippet_b = row.schema[:len_before_pronoun] + row.substitution_b.lower() + row.schema[-len_after_pronoun:]
    
    
    
    if row.correct_answer.lower() == 'a':
        return snippet_a, snippet_b
    return snippet_b, snippet_a

df['correct_sentence'], df['incorrrect_sentence'] = zip(*df.apply(generate_full_sentences, axis=1))

In [139]:
for index, row in df.iterrows():
    print(row.correct_sentence)

Os vereadores recusaram a autorização aos manifestantes porque os vereadores temiam a violência.
Os vereadores recusaram a autorização aos manifestantes porque os manifestantes eram favoráveis à violência.
A medalha não cabe na maleta porque a medalha é muito grande.
A medalha não cabe na maleta porque a maleta é muito pequena.
Joan certificou-se de agradecer Susan por toda ajuda que joan havia recebido.
Joan certificou-se de agradecer Susan por toda ajuda que joan havia oferecido.
Paul tentou ligar para o George, mas paul não foi bem sucedido.
Paul tentou ligar para o George, mas george não estava disponível.
O advogado fez uma pergunta ao juiz, mas ele estava relutante em repeti-la.o advogadoante em repeti-la.
O advogado fez uma perguta ao juiz, mas ele estava relutante em respondê-la.o juizte em respondê-la.
O caminhão de entregas passou rapidamente pelo ônibus escolar porque o caminhão de entregas estava indo muito depressa.
O caminhão de entregas passou rapidamente pelo ônibus esc

In [156]:
with open('data/processed/port_wsc.html', 'r') as f:
    soup = BeautifulSoup(f, 'html5lib')

rows = []
item = soup.find('ol').find_all('li', recursive=False)[40]
content = [it.strip().replace('  ', ' ') for it in item.text.split('\n') if it.strip() != '' ]
schema = (content[-6] + ' ' + content[-5])
snippet = content[-4].replace('Trecho: ', '')
if schema[-1] == '.' and schema[-1] != snippet[-1]:
    snippet += schema[-1]
pronoun = item.find('b').text.strip()
correct_answer = content[-1].replace('Resposta Correta: ', '').strip()[0]
substitution_a = content[-3]
substitution_b = content[-2]
row = [schema, snippet, pronoun, correct_answer, substitution_a, substitution_b]

row

['os punimos. Trecho: os',
 'punimos.',
 'os',
 'A',
 'Os alunos mais velhos',
 'Os alunos mais jovens']

In [157]:
item.decode()

'<li>\n            Os alunos mais velhos estavam fazendo bullying com os mais jovens, então nós\n            <b>os</b> punimos.<p></p><em> Trecho: </em>\n            <b>os</b>\n            punimos.\n            <p></p>\n            <ol type="A">\n                <li>Os alunos mais velhos </li>\n                <li>Os alunos mais jovens </li>\n            </ol>\n            <p><b> Resposta Correta: </b> A <br/><b> Fonte: </b> Ernest Davis </p>\n        </li>'

In [ ]:
# item.extract